# Step 0: Initial Imports

In [1]:
import sys
import os
import torch
from torch import optim
from time import time
import gdal
import numpy as np

In [2]:
#Dict for Mapping between CDL Int Label and String Correspondence:
CDL_dict = {0: "Background", 1: "Corn", 2: "Cotton", 3: "Rice", 4: "Sorghum", 5: "Soybeans", 6: "Sunflower", 10: "Peanuts", 11: "Tobacco", 12: "Sweet Corn", 13: "Pop or Orn Corn", 14: "Mint", 21: "Barley", 22: "Durum Wheat", 23: "Spring Wheat", 24: "Winter Wheat", 25: "Other Small Grains", 26: "Dbl Crop WinWht/Soybeans", 27: "Rye", 28: "Oats", 29: "Millet", 30: "Speltz", 31: "Canola", 32: "Flaxseed", 33: "Safflower", 34: "Rape Seed", 35: "Mustard", 36: "Alfalfa", 37: "Other Hay/Non Alfalfa", 38: "Camelina", 39: "Buckwheat", 41: "Sugarbeets", 42: "Dry Beans", 43: "Potatoes", 44: "Other Crops", 45: "Sugarcane", 46: "Sweet Potatoes", 47: "Misc Vegs & Fruits", 48: "Watermelons", 49: "Onions", 50: "Cucumbers", 51: "Chick Peas", 52: "Lentils", 53: "Peas", 54: "Tomatoes", 55: "Caneberries", 56: "Hops", 57: "Herbs", 58: "Clover/Wildflowers", 59: "Sod/Grass Seed", 60: "Switchgrass", 61: "Fallow/Idle Cropland", 63: "Forest", 64: "Shrubland", 65: "Barren", 66: "Cherries", 67: "Peaches", 68: "Apples", 69: "Grapes", 70: "Christmas Trees", 71: "Other Tree Crops", 72: "Citrus", 74: "Pecans", 75: "Almonds", 76: "Walnuts", 77: "Pears", 81: "Clouds/No Data", 82: "Developed", 83: "Water", 87: "Wetlands", 88: "Nonag/Undefined", 92: "Aquaculture", 111: "Open Water", 112: "Perennial Ice/Snow", 121: "Developed/Open Space", 122: "Developed/Low Intensity", 123: "Developed/Med Intensity", 124: "Developed/High Intensity", 131: "Barren", 141: "Deciduous Forest", 142: "Evergreen Forest", 143: "Mixed Forest", 152: "Shrubland", 176: "Grassland/Pasture", 190: "Woody Wetlands", 195: "Herbaceous Wetlands", 204: "Pistachios", 205: "Triticale", 206: "Carrots", 207: "Asparagus", 208: "Garlic", 209: "Cantaloupes", 210: "Prunes", 211: "Olives", 212: "Oranges", 213: "Honeydew Melons", 214: "Broccoli", 216: "Peppers", 217: "Pomegranates", 218: "Nectarines", 219: "Greens", 220: "Plums", 221: "Strawberries", 222: "Squash", 223: "Apricots", 224: "Vetch", 225: "Dbl Crop WinWht/Corn", 226: "Dbl Crop Oats/Corn", 227: "Lettuce", 229: "Pumpkins", 230: "Dbl Crop Lettuce/Durum Wht", 231: "Dbl Crop Lettuce/Cantaloupe", 232: "Dbl Crop Lettuce/Cotton", 233: "Dbl Crop Lettuce/Barley", 234: "Dbl Crop Durum Wht/Sorghum", 235: "Dbl Crop Barley/Sorghum", 236: "Dbl Crop WinWht/Sorghum", 237: "Dbl Crop Barley/Corn", 238: "Dbl Crop WinWht/Cotton", 239: "Dbl Crop Soybeans/Cotton", 240: "Dbl Crop Soybeans/Oats", 241: "Dbl Crop Corn/Soybeans", 242: "Blueberries", 243: "Cabbage", 244: "Cauliflower", 245: "Celery", 246: "Radishes", 247: "Turnips", 248: "Eggplants", 249: "Gourds", 250: "Cranberries", 254: "Dbl Crop Barley/Soybeans"}

In [3]:
#!sudo apt-get update
#!sudo apt-get install python-gdal -y

In [4]:
#Changing Path
tile2vec_dir = '/home/agupta21/gcloud/231n_gitproject'
sys.path.append('../')
sys.path.append(tile2vec_dir)

In [5]:
from src.datasets import TileTripletsDataset, GetBands, RandomFlipAndRotate, ClipAndScale, ToFloatTensor, triplet_dataloader
from src.tilenet import make_tilenet

In [6]:
from src.training import prep_triplets, train_triplet_epoch

In [7]:
#CHECK IF GPU IS ACTIVE AND ENABLED

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cuda = torch.cuda.is_available()
print(cuda)

False


In [9]:
#Testing a anchor/neighbor object (Model for reading in)

curNum = 1000
part1 = "../data/triplets/" + str(curNum)
anchor = np.load(part1 + "anchor.npy")
neighbor = np.load(part1 + "neighbor.npy")
distant = np.load(part1 + "distant.npy")

print(anchor.shape)

CDL_int1 = anchor[2][0][4] #print 3rd dimension, and last column of 0th row (same label for every example)
CDL_int2 = neighbor[2][0][4] 
CDL_int3 = distant[2][0][4]
print(CDL_dict[CDL_int1])
print(CDL_dict[CDL_int2])
print(CDL_dict[CDL_int3])


(50, 50, 5)
Winter Wheat
Winter Wheat
Grassland/Pasture


In [17]:
#Extract the label (5th dimension) and convert the array into 50 by 50 by 4.
anchor_label = anchor[2][0][4]
anchor_reshaped = anchor[:,:,0:4]
print(anchor_label, "Label of Tile")
print(anchor_reshaped.shape, "New shape")
#print(anchor[:,:,0:4])
#print(anchor.shape,anchor)

#visualize a tile

24 Label of Tile
(50, 50, 4) New shape


In [32]:
#Set up pipeline to link triplet labels to the various tiles 

labelToTiles = {}
import time
start = time.time()

for curNum in range(10000): # of triplets
    if curNum % 1000 == 0:
        print(curNum)
    #print(curNum)
    part1 = "../data/triplets/" + str(curNum)
    anchor = np.load(part1 + "anchor.npy")
    neighbor = np.load(part1 + "neighbor.npy")
    distant = np.load(part1 + "distant.npy")
    a_key = CDL_dict[anchor[2][0][4]]
    n_key = CDL_dict[neighbor[2][0][4]]
    d_key = CDL_dict[distant[2][0][4]]
    #print 3rd dimension, and last column of 0th row (same label for every example)
    labelToTiles.setdefault(a_key,[]).append(anchor)
    labelToTiles.setdefault(n_key,[]).append(neighbor)
    labelToTiles.setdefault(d_key,[]).append(distant)
    
end = time.time()
print(end - start)
ctr = 0
for key, value in labelToTiles.items():
    ctr += len(value)
    print(key,len(value))
print(ctr)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
483.39954566955566
Grapes 8619
Almonds 5021
Developed/Med Intensity 1704
Grassland/Pasture 4226
Dbl Crop WinWht/Corn 257
Oranges 337
Fallow/Idle Cropland 1939
Dbl Crop WinWht/Sorghum 38
Developed/Low Intensity 1172
Triticale 271
Developed/Open Space 1573
Alfalfa 1155
Woody Wetlands 41
Safflower 33
Tomatoes 445
Open Water 234
Winter Wheat 396
Developed/High Intensity 480
Cotton 271
Walnuts 360
Oats 57
Barley 52
Pistachios 708
Barren 62
Onions 73
Pomegranates 31
Shrubland 35
Other Tree Crops 11
Garlic 57
Corn 173
Rye 2
Citrus 12
Plums 20
Dry Beans 2
Olives 13
Durum Wheat 4
Carrots 26
Herbaceous Wetlands 2
Lettuce 5
Other Hay/Non Alfalfa 10
Cantaloupes 1
Sorghum 26
Dbl Crop Oats/Corn 8
Cherries 10
Pears 1
Peaches 10
Evergreen Forest 4
Deciduous Forest 4
Nectarines 2
Blueberries 3
Spring Wheat 1
Other Crops 1
Honeydew Melons 1
Watermelons 1
30000


In [ ]:
#all the values are unique (ie all grape tiles = different in the list)

# for i in range(5000):
#     for j in range(5000):
#         if i != j:
#             if np.array_equal(labelToTiles["Grapes"][i], labelToTiles["Grapes"][j]):
#                 print(i,j, "are equal")
